In [66]:
from estimate_underreporting import *
# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
df = pd.read_csv('sample_data/sample_reports_df.csv', parse_dates=['SRCreatedDate', 'InsCreatedDate', 'WOCreatedDate']).head(10000)
df.head(5)

,SRID,IncidentGlobalID,SRCreatedDate,InsCreatedDate,WOCreatedDate,SRCategory,BoroughCode,RiskRating
0,23071289,E5416B67-0692-4888-AB29-AD0391DF2C53,2022-02-19 21:55:00,2022-02-25 14:54:17,NaT,Remove Tree,Queens,7.0
1,23071289,E5416B67-0692-4888-AB29-AD0391DF2C53,2022-02-19 21:55:00,2022-02-25 14:54:17,NaT,Remove Tree,Queens,8.0
2,23493187,F3C21D50-E891-4829-8243-A817DA218747,2022-05-02 05:30:39,2022-05-03 15:19:22,2022-05-03 15:20:12,Hazard,Bronx,9.0
3,24258242,1F58426B-0BB3-40DB-BFCE-8FC2341CB9EA,2022-08-15 17:35:43,2022-08-22 16:42:07,2022-08-22 16:44:17,Prune,Queens,8.0
4,24258242,1F58426B-0BB3-40DB-BFCE-8FC2341CB9EA,2022-08-15 17:35:43,2022-08-22 16:36:06,2022-08-22 16:38:14,Prune,Queens,NaN


In [72]:
incidents = create_incidents_df(df,
                        incident_identifier_col = ['IncidentGlobalID'], 
                        observation_start_col = ['SRCreatedDate'],
                        observation_end_col = ['InsCreatedDate', 'WOCreatedDate'], 
                        covariates_cont = ['RiskRating'],
                        covariates_cat = ['SRCategory', 'BoroughCode'])
y, X, duration = prepare_data_for_regression(incidents, 
                                             covariate_cat = ['SRCategory', 'BoroughCode'],
                                             covariate_cont = ['RiskRating'],)
res_standard = train_model(y, X, duration, model_type='standard')

Optimization terminated successfully.
         Current function value: 0.545703
         Iterations: 128
         Function evaluations: 129
         Gradient evaluations: 129
                          Poisson Regression Results                          
Dep. Variable:         num_duplicates   No. Observations:                 4441
Model:                        Poisson   Df Residuals:                     4427
Method:                           MLE   Df Model:                           13
Date:                Wed, 04 Oct 2023   Pseudo R-squ.:                  0.1009
Time:                        18:13:05   Log-Likelihood:                -2423.5
converged:                       True   LL-Null:                       -2695.3
Covariance Type:            nonrobust   LLR p-value:                7.666e-108
                                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [74]:
res_zeroinflated = train_model(y, X, duration, model_type='zeroinflated')

Optimization terminated successfully.
         Current function value: 0.473045
         Iterations: 126
         Function evaluations: 127
         Gradient evaluations: 127
                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:          num_duplicates   No. Observations:                 4441
Model:             ZeroInflatedPoisson   Df Residuals:                     4427
Method:                            MLE   Df Model:                           13
Date:                 Wed, 04 Oct 2023   Pseudo R-squ.:                 0.08881
Time:                         18:14:05   Log-Likelihood:                -2100.8
converged:                        True   LL-Null:                       -2305.5
Covariance Type:             nonrobust   LLR p-value:                 2.188e-79
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------

In [76]:
incidents = append_delays_to_incidents_df(incidents,
                                          generate_predicted_reporting_delay(res_standard, X))

In [77]:
incidents

,IncidentGlobalID,num_duplicates,duration,RiskRating,SRCategory,BoroughCode,predicted_reporting_delay
0,00019105-8135-4AC1-A4B7-003C7793EB03,0,4.971748,0.101346,Remove Tree,Bronx,174.447055
1,0003AE41-2F12-4AB3-9452-5C9AA6BA1F37,0,3.918750,0.101346,Hazard,Queens,21.921806
2,0007E6E1-65AF-4CC5-88EA-F814B84EE3A4,0,0.128056,0.101346,Hazard,Queens,21.921806
3,0008BE00-79E8-4DF7-8636-D1409B011957,0,0.554387,-1.040979,Remove Tree,Queens,136.949451
4,000AAB1F-254E-41AC-A8E9-635A95A3619C,0,4.906019,-0.469817,Hazard,Staten Island,33.898782
...,...,...,...,...,...,...,...
5969,FF8C934D-0F76-4483-9B09-992B73A830CC,0,0.613889,1.243670,Hazard,Queens,16.271405
5970,FFC7227E-044A-43FC-B2C5-FC8E73C7C58C,0,3.187095,0.672508,Hazard,Queens,18.886466
5971,FFCD3B76-0715-4F72-B851-4D5C8E0B7334,0,1.009942,1.243670,Hazard,Queens,16.271405
5972,FFCE86B9-41E8-4E49-BD3A-5D10B0DB8E0A,0,1.995451,0.101346,Hazard,Brooklyn,13.633316
